# Constants

In [1]:
OUR_AIRCRAFT_ID = 'ally_1'

# Libraries

In [2]:
import numpy as np

import os

import pandas as pd

import sqlite3

# Import Data

In [3]:
my_dir = os.path.join('..', 'data', 'v1_logs.db')

# Connect to the database
conn = sqlite3.connect(my_dir)

aircraft_states = pd.read_sql_query("SELECT * FROM aircraft_states", conn)

aircraft_adjustments = pd.read_sql_query("SELECT * FROM aircraft_adjustments", conn)

# Close the connection
conn.close()

# Preprocess

## Split array-like fields

In [4]:
COLS = ['position', 'Euler_angles', 'move_vector']
for col in COLS:

    df_0 = aircraft_states.copy()

    if not (col in df_0.columns):
        print(f'Column {col} does not exist.')
        continue
        
    df_1 = df_0\
        .drop(columns=col)\
        .copy()
    
    df_2 = df_0\
        .loc[:, col]\
        .str\
        .replace('[', '')\
        .str\
        .replace(']', '')\
        .str\
        .split(',', expand=True)\
        .astype(np.float64)\
        .rename(columns={0: f'{col}_x', 1: f'{col}_y', 2: f'{col}_z'})\
        .copy()

    aircraft_states = pd.concat([df_1, df_2], axis=1)

## Drop features

In [5]:
COLS = [
    'timestep',
    'crashed',
    'destroyed',
    'easy_steering',
    'active',
    'type',
    'nationality',
    'horizontal_speed',
    'vertical_speed',
    'linear_speed',
    'linear_acceleration',
    'gear',
    'ia',
    'autopilot',
    'autopilot_heading',
    'autopilot_speed',
    'autopilot_altitude',
    'target_id',
    'log_timestamp',
    'heading',
    'altitude',
    'pitch_attitude',
    'roll_attitude',
    'flaps_level',
    'target_angle',
    'user_pitch_level',
    'user_roll_level',
    'user_yaw_level'
]
aircraft_states = aircraft_states\
    .drop(columns=COLS, errors='ignore')\
    .copy()

## Pair our arcraft with bandit

In [6]:
df_0 = aircraft_states.copy()
# get states of our aircraft
df_1 = df_0\
    .query('log_machine_id == @OUR_AIRCRAFT_ID')\
    .drop(columns=['log_machine_id'])\
    .copy()

# exclude features that we can't get from the bandit.
df_2 = df_0\
    .query('log_machine_id != @OUR_AIRCRAFT_ID')\
    .drop(columns=[
        'log_machine_id', 'timestamp', 'health_level', 'wreck',
        'thrust_level', 'brake_level',
        'post_combustion', 'target_locked', 'target_out_of_range',])\
    .copy()

# merge
df_3 = df_1.merge(
    df_2,
    left_on=['simulation_id', 'log_id'],
    right_on=['simulation_id', 'log_id'],
    how='inner',
    suffixes=(None, '_bandit'))\
    .reset_index(drop=True)\
    .copy()

preprocessed = df_3.copy()

## Join aircraft adjustments

In [7]:
preprocessed.columns

Index(['timestamp', 'health_level', 'wreck', 'thrust_level', 'brake_level',
       'post_combustion', 'target_locked', 'target_out_of_range',
       'simulation_id', 'log_id', 'position_x', 'position_y', 'position_z',
       'Euler_angles_x', 'Euler_angles_y', 'Euler_angles_z', 'move_vector_x',
       'move_vector_y', 'move_vector_z', 'position_x_bandit',
       'position_y_bandit', 'position_z_bandit', 'Euler_angles_x_bandit',
       'Euler_angles_y_bandit', 'Euler_angles_z_bandit',
       'move_vector_x_bandit', 'move_vector_y_bandit', 'move_vector_z_bandit'],
      dtype='object')

In [ ]:
preprocessed.merge(
    aircraft_adjustments,
    left_on=['simulation_id', 'log_id'],
    right_on=['simulation_id', 'log_id'],
    how='inner')

In [ ]:
.columns

## Build unique timestamp

In [ ]:
srs = preprocessed.loc[:, 'simulation_id'] + preprocessed.loc[:, 'timestamp']
srs = pd.to_datetime(srs, unit='s')
preprocessed.drop(
    columns=['simulation_id', 'timestamp'],
    inplace=True)
preprocessed.loc[:, 'timestamp'] = srs
preprocessed = preprocessed\
    .set_index('timestamp')\
    .reset_index(drop=False)

# Analysis

## Calculate reward

In [ ]:
preprocessed